In [5]:
# Importing all the required packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [6]:
# Importing the dataset
# Converting them to dataframes

books = pd.read_csv('Dataset/BX-Books.csv',delimiter = ";",on_bad_lines = 'skip', encoding='latin-1')
books.head()

/var/folders/7b/x84bg1957vdds256_wq6x6h40000gn/T/ipykernel_53081/2689315079.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Dataset/BX-Books.csv',delimiter = ";",on_bad_lines = 'skip', encoding='latin-1')


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [7]:
# There are various sizes of the book covers
# I'll be using the large size image for the website
# Feature Selection: I will remove the columns Image-URL-S, Image-URL-M and

books = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication','Publisher','Image-URL-L']]
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [8]:
# The column names in the csv file are long
# So, I am shortening them to something simple

books.rename(columns={
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-UNL-L":"img_url"},inplace = True)

In [9]:
books.head()

,ISBN,title,author,year,publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [10]:
# Importing the BX-Users.csv

users = pd.read_csv('Dataset/BX-Users.csv',delimiter = ";",on_bad_lines = 'skip', encoding='latin-1')
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
ratings = pd.read_csv('Dataset/BX-Book-Ratings.csv',delimiter = ";",on_bad_lines = 'skip', encoding='latin-1')
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [12]:
# Editing the names of the columns in the rating csv

ratings.rename(columns = {
    "User-ID":"user_id",
    "Book-Rating":"rating"
},inplace = True)

ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [13]:
# Creating a boolean series x (condition is: any user that has rated more than 20 books)

x = ratings['user_id'].value_counts() > 20

# Boolean indexing uses this boolean Series to filter the original Series or DataFrame
# It selects rows or elements where the corresponding boolean value is True

y = x[x]

# Filtered out all the people that have more than 20 book ratings

ratings[ratings['user_id'].isin(y)]

ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [14]:
# Merging the ratings with the books
# This would store the ratings and book names in the same dataframe

ratings_with_books = ratings.merge(books,on="ISBN")
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...


In [15]:
# Grouping the books with the number of ratings
# When groupby is performed (or any other aggregation), 
# the resulting dataframe has no index at all
# reset_index moves the current index
# into columns and resets the index to the default integer index (0, 1, 2, ...).

num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()
num_rating.head()

# Renamed rating to number_of_rating

num_rating.rename(columns={'rating':'number_of_rating'},inplace=True)

# Mergin the ratings with number of ratings
# This is to filter out all the books that have more than 20 ratings
# The lesser rating books might not give accurate recommendations

final_rating = ratings_with_books.merge(num_rating,on="title")
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-L,number_of_rating
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60


In [16]:
# Now, filtering out all the books that have more than 30 ratings

final_rating = final_rating[final_rating['number_of_rating'] >= 50]
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,Image-URL-L,number_of_rating
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,60


In [17]:
# Now, we need to drop all the duplicate values of title and user_id

final_rating.drop_duplicates(['user_id','title'],inplace = True)
final_rating.shape

(285518, 9)

In [18]:
# Making the pivot table for clustering
# Filling all the null values to 0
# This is logical, cuz the user might not have read the book
# The non reading people would be in the same cluster

book_pivot = final_rating.pivot_table(columns='user_id',index='title',values='rating')
book_pivot.fillna(0,inplace=True)
book_pivot.head()

user_id,8,9,14,16,17,19,23,26,32,39,...,278820,278824,278828,278832,278836,278843,278844,278846,278851,278854
title,,,,,,,,,,,,,,,,,,,,,
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Handling the zeroes

book_sparse = csr_matrix(book_pivot)
book_sparse

<2444x47994 sparse matrix of type '<class 'numpy.float64'>'
	with 113444 stored elements in Compressed Sparse Row format>

In [20]:
# Declaring an instance of the model
# I am using the Nearest Neighbour algorithm for clutering

model = NearestNeighbors(algorithm = "brute")

In [21]:
# Fiting the pivot table (dataset) to the model

model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [22]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=8)
distance

array([[ 0.        , 73.70888685, 74.13501197, 74.14849965, 74.25631286,
        74.42445835, 74.43789358, 74.4848978 ]])

In [23]:
suggestion

array([[ 237, 1391, 1713,  770,   65,  284, 2423,  679]])

In [24]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Beloved (Plume Contemporary Fiction)', 'Reasonable Doubt',
       'The Blooding', 'Ground Zero and Beyond', 'A Secret Affair',
       'Blood and Gold (Rice, Anne, Vampire Chronicles.)', 'Women in His Life',
       'Flight of the Old Dog'],
      dtype='object', name='title')


In [25]:
books_name = book_pivot.index

In [27]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=8)
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [29]:
book_name = 'Harry Potter and the Prisoner of Azkaban (Book 3)'
recommend_book(book_name)

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Chamber of Secrets (Book 2)
All I Need Is You
Eternity
Blood and Gold (Rice, Anne, Vampire Chronicles.)
Ground Zero and Beyond
The Mouse and the Motorcycle
